In [ ]:
library(Matrix)
library(Seurat)

seurat_object <- readRDS("./scRNA-seq/20230911_tonsil_atlas_rna_seurat_obj.rds")
output_dir <- "./10x_format_output/"
if (!dir.exists(output_dir)) dir.create(output_dir)

# Ensure sparse dgCMatrix
counts <- as(seurat_object@assays$RNA@counts, "dgCMatrix")

# ---- write plain files ----

barcodes_file <- file.path(output_dir, "barcodes.tsv")
features_file <- file.path(output_dir, "features.tsv")
matrix_file   <- file.path(output_dir, "matrix.mtx")

write.table(
  colnames(counts),
  barcodes_file,
  quote = FALSE,
  row.names = FALSE,
  col.names = FALSE
)

# features.tsv (THREE columns — required)
features <- data.frame(
  gene_id = rownames(counts),
  gene_name = rownames(counts),
  feature_type = "Gene Expression"
)

write.table(
  features,
  features_file,
  sep = "\t",
  quote = FALSE,
  row.names = FALSE,
  col.names = FALSE
)

writeMM(counts, matrix_file)

# ---- gzip safely (no redirection) ----

system2("gzip", c("-f", barcodes_file))
system2("gzip", c("-f", features_file))
system2("gzip", c("-f", matrix_file))

# ---- metadata ----

write.csv(
  seurat_object@meta.data,
  file = file.path(output_dir, "seurat_observations.csv"),
  row.names = TRUE
)